In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=07002785227c4a43cadc85029d3c0e86ec6a7be9e4f1a2ed690e89b4c6d4f14c
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU
def printm():
  GPUs = GPU.getGPUs()
  gpu = GPUs[0]
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

In [ ]:
printm()

IndexError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['KAGGLE_USERNAME']="daominhkhanh"
os.environ['KAGGLE_KEY']="90e6f266906e9dcfeae1afea9a0630ac"
!kaggle datasets download -d hsankesara/flickr-image-dataset

100% 8.16G/8.16G [03:08<00:00, 25.5MB/s]
100% 8.16G/8.16G [03:08<00:00, 46.6MB/s]


In [ ]:
!unzip /content/flickr-image-dataset.zip
!mkdir Flickr30k
!rm /content/flickr-image-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5354911151.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/535529555.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5356236664.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5356318668.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/535830521.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5359437945.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5361355399.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/53614287.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/536147209.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5361551308.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/536156200.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/

In [ ]:
!mv /content/flickr30k_images/flickr30k_images /content/Flickr30k
!mv /content/flickr30k_images/results.csv /content/Flickr30k


In [ ]:
!cp /content/drive/MyDrive/Flickr30k/captions_train.csv /content/Flickr30k
!cp /content/drive/MyDrive/Flickr30k/captions_test.csv /content/Flickr30k
!cp /content/drive/MyDrive/Flickr30k/captions_val.csv /content/Flickr30k

In [ ]:
printm()

Gen RAM Free: 12.7 GB  | Proc size: 129.7 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


In [ ]:
import torch
from torch import optim
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import transforms as T
import pickle
from torch import nn
import numpy as np 
from PIL import Image 
import pandas as pd 
import time

In [ ]:
import os
import torch
from torchvision import transforms as T
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import pandas as pd
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.text import Tokenizer

class Vocabulary:
    def __init__(self):
        self.tokenizer=Tokenizer(num_words=9000,
                                filters='!"#$%&()*+/:;,"=?@[\\]^_`{|}~\t\n',
                                lower=True,
                                oov_token='<unk>'
        )

    def __len__(self):
        return len(self.tokenizer.word_index)

    def build_vocab(self,captions_list):
        self.tokenizer.fit_on_texts(captions_list)
        self.sequences=self.tokenizer.texts_to_sequences(captions_list)
        self.tokenizer.word_index['<pad>']=0
        self.tokenizer.index_word[0]='<pad>'   
        self.stoi=self.tokenizer.word_index
        self.itos=self.tokenizer.index_word

    def numericalize(self,text):
        return self.tokenizer.texts_to_sequences([text])[0]


class Flickr8k(Dataset):
    def __init__(self,root_dir,caption_file,vocabulary=None,transforms=None):
        path=os.getcwd()
        self.transforms=transforms
        path_to_caption_file=os.path.join(path,root_dir+"/"+caption_file)
        df=pd.read_csv(path_to_caption_file,sep='|',skipinitialspace=True)
        self.images=df.image_name
        self.captions='<sos> '+df.comment+' <eos>'
        self.captions=self.captions.tolist()
        if vocabulary is None:
          self.seq_exist=True
          self.vocabulary=Vocabulary()
          self.vocabulary.build_vocab(self.captions)
        else:
          self.vocabulary=vocabulary
          self.seq_exist=False

    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        image_name=self.images[idx]
        path=os.path.join(os.getcwd(),"Flickr30k/flickr30k_images/"+str(image_name))
        image=Image.open(path).convert("RGB")
        if self.transforms is not None:
            image=self.transforms(image)
        if self.seq_exist is True:
          caption_vector=self.vocabulary.sequences[idx]
        else:
          caption=self.captions[idx]
          caption_vector=self.vocabulary.numericalize(caption)
        return image,torch.tensor(caption_vector)


class MyCollate:
    def __init__(self,pad_idx):
        self.pad_idx=pad_idx

    def __call__(self,batch):
        #batch has a lot of pait (image,caption)      
        images=[item[0].unsqueeze(dim=0) for item in batch]
        captions=[item[1] for item in batch]
        images=torch.cat(images,dim=0)#batch_size*3*height*width
        captions=pad_sequence(captions,batch_first=True,padding_value=self.pad_idx)
        return images,captions



In [ ]:
transforms=T.Compose([
        T.Resize(226),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225))
    ])

In [ ]:
dataset=Flickr8k(root_dir='Flickr30k',
                 caption_file='captions_train.csv',
                 transforms=transforms
                )
pad_idx=dataset.vocabulary.stoi['<pad>']
data_train=DataLoader(dataset=dataset,
                        batch_size=32,
                        num_workers=2,
                        shuffle=True,
                        pin_memory=True,#if pin_memory=True, which enables fast data transfer to CUDA-enable GPUS
                        collate_fn=MyCollate(pad_idx)
)


In [ ]:
val=Flickr8k(root_dir='Flickr30k',
                 caption_file='captions_val.csv',
                 transforms=transforms,
                 vocabulary=dataset.vocabulary
                )
data_val=DataLoader(dataset=val,
                    batch_size=32,
                    num_workers=2,
                    shuffle=True,
                    pin_memory=True,#if pin_memory=True, which enables fast data transfer to CUDA-enable GPUS
                    collate_fn=MyCollate(pad_idx)
)

In [ ]:
with open('/content/drive/MyDrive/Flickr30k/dataset.pickle','wb') as file:
    pickle.dump(dataset,file,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
test=pd.read_csv('Flickr30k/captions_test.csv',sep='|',skipinitialspace=True)

In [ ]:
import torch
from torch import nn
from torchvision import models
from torch.nn import functional
from torch.autograd import Variable
import numpy as np 

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Encoder(nn.Module):
    def __init__(self,units=512,encode_image_size=14):
        super(Encoder,self).__init__()
        self.resnet=models.resnet101(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad=False

        modules=list(self.resnet.children())[:-2]
        self.resnet=nn.Sequential(*modules)
        self.adap=nn.AdaptiveAvgPool2d((encode_image_size,encode_image_size))


    def forward(self,images):
        features=self.resnet(images)
        features=self.adap(features)
        features=features.permute(0,2,3,1)
        features=features.view(features.size(0),-1,features.size(-1))#batch_size*num_pixels*encoder_dim
        return features

class AttentionBahdanau(nn.Module):
    def __init__(self,attention_dim,encoder_dim,decoder_dim):
        super(AttentionBahdanau,self).__init__()
        self.attention_dim=attention_dim
        self.encoder_dim=encoder_dim
        self.decoder_dim=decoder_dim

        self.weight_decoder=nn.Linear(decoder_dim,attention_dim)
        self.weight_encoder=nn.Linear(encoder_dim,attention_dim)
        self.full=nn.Linear(attention_dim,1)
        self.relu=nn.ReLU()
        self.softmax=nn.Softmax(dim=1)

    def forward(self,encoder_out,hidden_decoder):
        #encoder_out: batch_size*num_pixel*encoder_dim
        #hidden_decoder: batch_size*decoder_dim

        energies=self.full(
                torch.tanh(
                    self.weight_encoder(encoder_out)+self.weight_decoder(hidden_decoder).unsqueeze(1)
                    )
            )#batch_size*num_pixels*1
        energies=energies.squeeze(dim=2)
        attention_weights=self.softmax(energies)#batch_size*num_pixels
        context_vector=(encoder_out*attention_weights.unsqueeze(dim=2)).sum(dim=1)#batch_size*encoder_dim
        return context_vector,attention_weights


class Decoder(nn.Module):
    def __init__(self,attention_dim,decoder_dim,embedding_dim,vocab_size,vocabulary,encoder_dim=2048,drop_pro=0.2):
        super(Decoder,self).__init__()
        self.attention_dim=attention_dim
        self.encoder_dim=encoder_dim
        self.decoder_dim=decoder_dim
        self.embedding_dim=embedding_dim
        self.vocab_size=vocab_size
        self.vocabulary=vocabulary

        self.embedding=nn.Embedding(vocab_size,embedding_dim)
        self.attention=AttentionBahdanau(attention_dim,encoder_dim,decoder_dim)
        self.lstm_cell1=nn.LSTMCell(input_size=embedding_dim+encoder_dim,
                        hidden_size=decoder_dim,
                        bias=True
        )   

        self.lstm_cell2=nn.LSTMCell(input_size=decoder_dim,
                        hidden_size=decoder_dim,
                        bias=True
        )  

        # self.lstm_cell3=nn.LSTMCell(input_size=decoder_dim,
        #                 hidden_size=decoder_dim,
        #                 bias=True
        # )   

        self.fcn=nn.Linear(decoder_dim,vocab_size)
        self.dropout=nn.Dropout(drop_pro)
        self.init_h=nn.Linear(encoder_dim,decoder_dim)
        self.init_c=nn.Linear(encoder_dim,decoder_dim)
        self.init_weights()
    
    # def load_pretrained_embedding(self):
    #     embedding_dict = dict()
    #     fid = open('glove.6B.100d.txt' ,encoding="utf8")
    #     for line in fid:
    #         values = line.split()
    #         word = values[0]
    #         coefs = np.asarray(values[1:], dtype='float32')
    #         embedding_dict[word] = coefs
    #     fid.close()
        
    #     embedding_matrix=torch.zeros(self.vocab_size,self.embedding_dim)
    #     for word,idx in self.vocabulary.stoi.items():
    #         embedding_vector=embedding_dict.get(word)
    #         if embedding_vector is not None:
    #             embedding_matrix[idx]=embedding_vector
        
    #     return embedding_matrix

    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fcn.bias.data.fill_(0)
        self.fcn.weight.data.uniform_(-0.1,0.1)

    def load_embedding_pretrained(self,embeddings):
        self.embedding.weight=nn.Parameter(embeddings)
    
    def fine_tune_embeddings(self,fine_tune=True):
        for param in self.embedding.parameters():
            param.requires_grad=fine_tune

    def init_hidden_state(self,features):
        mean_features=features.mean(dim=1)
        h1=self.init_h(mean_features).to(device)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
        c1=self.init_c(mean_features).to(device)
        batch_size=features.size(0)
        h2=torch.zeros(batch_size,self.decoder_dim).to(device)
        c2=torch.zeros(batch_size,self.decoder_dim).to(device)
        return h1,c1,h2,c2

    def forward(self,features,captions):
        #features: batch_size*num_pixels*encoder_dim
        #captions: batch_size*seq_length
        embedded=self.embedding(captions)#batch_size*seq_length*embedding_size
        seq_length=captions.size(1)-1
        batch_size=captions.size(0)
        h1,c1,h2,c2=self.init_hidden_state(features)
        num_pixels=features.size(1)

        preds=torch.zeros(batch_size,seq_length,self.vocab_size).to(device)
        attention_weights=torch.zeros(batch_size,seq_length,num_pixels).to(device)

        for i in range(seq_length):
            #context_vectore:batch_size*encoder_dim
            #attention_weight:batch_size*num_pixel
            context_vector,attention_weight=self.attention(features,h2)
            attention_weights[:,i,:]=attention_weight
            lstm_input=torch.cat((embedded[:,i,:],context_vector),dim=1)
            h1,c1=self.lstm_cell1(lstm_input,(h1,c1))#batch_size*decoder_dim
            h2,c2=self.lstm_cell2(h1,(h2,c2))
            outputs=self.fcn(self.dropout(h2))#batch_size*vocab_size
            preds[:,i,:]=outputs
        return preds,attention_weights


    def image_captions(self,feature,vocabulary,max_length=30):
        start_word=torch.tensor(vocabulary.stoi['<sos>']).view(1,-1).to(device)
        embedding=self.embedding(start_word)#1*batch_size*embedding_size
        h1,c1,h2,c2=self.init_hidden_state(feature)
        attention_weights=[]
        caption=[]
        for _ in range(max_length):
            context,attention_weight=self.attention(feature,h2)
            attention_weights.append(attention_weight)
            lstm_in=torch.cat((embedding[:,0,:],context),axis=1)
            h1,c1=self.lstm_cell1(lstm_in,(h1,c1))#batch_size*decoder_dim
            h2,c2=self.lstm_cell2(h1,(h2,c2))
            outputs=self.fcn(self.dropout(h2))
            predict_word=outputs.argmax(dim=1)
            if vocabulary.itos[predict_word.item()]=="<eos>":
                break
            
            caption.append(predict_word.item())
            embedding=self.embedding(predict_word.unsqueeze(dim=0))

        return [vocabulary.itos[index] for index in caption]
    
    # def image_caption_with_beam_search(self,feature,vocabulary,beam_size=3,max_length=30):
    #         h1,c1=self.init_hidden_state(feature)
    #         words=[[[h1,c1],[[vocabulary.stoi['<sos>']],0.0]]]
    #         for i in range(max_length):
    #             temp=[]
    #             for idx,(hidden_state,word) in enumerate(words):
    #                 current_word=word[0][-1]
    #                 embedding=self.embedding(torch.tensor(current_word).view(1,-1).to(device))
    #                 h1_s,c1_s=hidden_state
    #                 context_vector,_=self.attention(feature,h1_s)
    #                 lsmt_in=torch.cat((embedding[:,0,:],context_vector),dim=1)
    #                 h1_s,c1_s=self.lstm_cell1(lsmt_in,(h1_s,c1_s))
    #                 outputs=self.fcn(self.dropout(h1_s)).cpu().numpy()
    #                 word_preds=np.argsort(outputs[0])[-beam_size:]
                

    #                 for index in word_preds:
    #                     cap_current,prob=word[0][:],word[1]
    #                     cap_current.append(index)
    #                     prob+=outputs[0][index]
    #                     temp.append([[h1_s,c1_s],[cap_current,prob]])

    #             words=temp
    #             words=sorted(words,reverse=False,key=lambda l:l[1][1])
    #             words=words[-beam_size:]
            
    #         final_prediction=words[-1][1][0]
    #         captions=[]
    #         for index in final_prediction:
    #             if vocabulary.itos[index]!="<eos>":
    #                 captions.append(vocabulary.itos[index])
    #             else:
    #                 break

    #         return captions
        

In [ ]:
vocab_size=len(dataset.vocabulary)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=300
attention_dim=512
decoder_dim=512
embedding_dim=256
encoder=Encoder().to(device)
encoder_optim=optim.RMSprop(encoder.parameters(),lr=1e-4)
decoder=Decoder(attention_dim=attention_dim,decoder_dim=decoder_dim,embedding_dim=embedding_dim,vocab_size=vocab_size,vocabulary=dataset.vocabulary).to(device)
decoder_optim=optim.RMSprop(decoder.parameters(),lr=4e-4)
criterion=nn.CrossEntropyLoss()

In [ ]:
with open('/content/drive/MyDrive/Flickr30k/dataset.pickle','rb') as file:
  dataset=pickle.load(file)

In [ ]:
vocabulary=dataset.vocabulary

In [ ]:
state_dict=torch.load('/content/drive/MyDrive/Flickr30k/model...pth')
attention_dim=state_dict['attention_dim']
embedding_dim=state_dict['embedding_dim']
decoder_dim=state_dict['decoder_dim']
vocab_size=state_dict['vocab_size']
encoder=Encoder().to(device)
decoder=Decoder(attention_dim=attention_dim,decoder_dim=decoder_dim,embedding_dim=embedding_dim,vocab_size=vocab_size,vocabulary=dataset.vocabulary).to(device)
encoder.load_state_dict(state_dict['encoder_state_dict'])
decoder.load_state_dict(state_dict['decoder_state_dict'])

In [ ]:
from random import randint
from matplotlib import pyplot as plt 
from bleu import sentence_bleu,SmoothingFunction
smooth=SmoothingFunction()

def save_loss(loss_train_history,loss_val_history,epoch):
    train=np.asarray(loss_train_history)
    val=np.asarray(loss_val_history)
    np.save('/content/drive/MyDrive/Flickr30k/train{}.npy'.format(epoch),train)
    np.save('/content/drive/MyDrive/Flickr30k/val{}.npy'.format(epoch),val)
    print("save loss done")


def preprocessing(image_name):
    image = Image.open("/content/Flickr30k/flickr30k_images/"+image_name)
    image=transforms(image)
    return image

def get_list_caption(image_name):
    vocabulary=dataset.vocabulary
    indxs=np.where(test.image_name==image_name)[0]
    result=[]
    for index in indxs:
        s=str(test.iloc[index]['comment'])
        s=s.strip()
        result.append(s.split(' '))

    return result

def show_image(image,caption,epoch):
    image[0]=image[0]*0.229+0.485
    image[1]=image[1]*0.224+0.456
    image[2]=image[2]*0.225+0.406
    image=image.cpu().numpy().transpose((1,2,0))
    fig,ax=plt.subplots(1,1)
    img=ax.imshow(image,interpolation='nearest')
    if caption is not None:
        ax.set_title('Greedy:'+caption)
    plt.savefig('/content/drive/MyDrive/Flickr30k/image{}.png'.format(epoch))
    print("save image done")

def evaluate_bleu_score_test_set(epoch):
    encoder.eval()
    decoder.eval()
    bleu1,bleu2,bleu3,bleu4=0,0,0,0
    images=set(test.image_name)
    file=open('/content/drive/MyDrive/Flickr30k/caption_result{}.txt'.format(epoch),'w')
    file.write('image,captionsn\n')
    t=randint(0,len(images))
    with torch.no_grad():
      for idx,image_name in enumerate(images):
          image=preprocessing(image_name)
          image=image.to(device)
          feature=encoder(image.unsqueeze(dim=0))
          caption_greedy=decoder.image_captions(feature,dataset.vocabulary)
          list_caption=get_list_caption(image_name)
          cap=' '.join([word for word in caption_greedy])
          if idx==t:
            print(image_name)
            print(cap)
            show_image(image,cap,epoch)

          file.write(f'{image_name},{cap}\n')
          bleu1+=sentence_bleu(list_caption,caption_greedy,weights=(1,0,0,0))
          bleu2+=sentence_bleu(list_caption,caption_greedy,weights=(0.5,0.5,0,0))
          bleu3+=sentence_bleu(list_caption,caption_greedy,weights=(0.333,0.333,0.333,0))
          bleu4+=sentence_bleu(list_caption,caption_greedy,weights=(0.25,0.25,0.25,0.25))

    file.close()
    print("Greedy seach:")
    print("Bleu for 1-gram:",bleu1*1.0/len(images))
    print("Bleu for 2-gram:",bleu2*1.0/len(images))
    print("Bleu for 3-gram:",bleu3*1.0/len(images))
    print("Bleu for 4-gram:",bleu4*1.0/len(images))



def save_model(epoch):
    model_state={
        'epochs':epochs,
        'current_epoch':epoch,
        'attention_dim':attention_dim,
        'decoder_dim':decoder_dim,
        'embedding_size':embedding_dim,
        'vocab_size':vocab_size,
        'encoder_state_dict':encoder.state_dict(),
        'decoder_state_dict':decoder.state_dict()
    }
    torch.save(model_state,"/content/drive/MyDrive/Flickr30k/model{}.pth".format(epoch))
    print("save model done")

def evaluate():
    encoder.eval()
    decoder.eval()
    loss_total=0
    for idx,(images,captions) in enumerate(data_val):
        images=images.to(device)
        captions=captions.to(device)
        features=encoder(images)
        outputs,_=decoder(features,captions)
        loss=criterion(outputs.reshape(-1,vocab_size),captions[:,1:].reshape(-1))
        loss_total+=loss.item()
    return loss_total/len(data_val)




In [ ]:
print(len(data_train))

1563


In [ ]:
import time
loss_train_history=[]
loss_val_history=[]
encoder.train()
decoder.train()
for epoch in range(0,epochs+1):
    start_time=time.time()
    encoder.train()
    decoder.train()
    for idx,(images,captions) in enumerate(data_train):
          images=images.to(device)
          captions=captions.to(device)
          features=encoder(images)
          outputs,attention_weights=decoder(features,captions)
          targets=captions[:,1:]
          loss=criterion(outputs.reshape(-1,vocab_size),targets.reshape(-1))
          encoder_optim.zero_grad()
          decoder_optim.zero_grad()
          loss.backward()
          encoder_optim.step()
          decoder_optim.step()
          if idx%100==0:
            print(idx,end=' ')
    loss_train_history.append(loss.item())
    printm()
    loss_val=evaluate()
    loss_val_history.append(loss_val)
    print("Epoch:{}----Train:{}---Val:{}----Run:{}s".format(epoch,loss.item(),loss_val,-start_time+time.time()))
    if epoch!=0 and epoch %5==0:
        save_loss(loss_train_history,loss_val_history,epoch)
        save_model(epoch)
        evaluate_bleu_score_test_set(epoch) 

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 Gen RAM Free: 10.8 GB  | Proc size: 4.0 GB
GPU RAM Free: 7467MB | Used: 7642MB | Util  51% | Total 15109MB
Epoch:0----Train:2.344266414642334---Val:1.8457215701593266----Run:987.2223029136658s
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 Gen RAM Free: 10.8 GB  | Proc size: 4.0 GB
GPU RAM Free: 5911MB | Used: 9198MB | Util  61% | Total 15109MB
Epoch:1----Train:1.8428137302398682---Val:1.695867406788157----Run:989.8142685890198s
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 Gen RAM Free: 10.8 GB  | Proc size: 4.0 GB
GPU RAM Free: 5773MB | Used: 9336MB | Util  62% | Total 15109MB
Epoch:2----Train:1.5222870111465454---Val:1.6529974654030952----Run:991.8796482086182s
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 Gen RAM Free: 10.8 GB  | Proc size: 4.0 GB
GPU RAM Free: 4475MB | Used: 10634MB | Util  70% | Total 15109MB
Epoch:3----Train:1.5423892736434937---Va